<h1> V02 </h1>

In [7]:
# # -*- coding: utf-8 -*-
# """
# Final Data Generator (version 2)
# Structure:
# datasets/
#    ep_000/
#       light/
#       moderate/
#       heavy/
#       dataset_metadata.json
# """

# from __future__ import annotations
# from dataclasses import dataclass, asdict, replace
# from typing import List, Dict, Optional
# import numpy as np
# import pandas as pd
# import os, json, math, time, platform, getpass
# import matplotlib.pyplot as plt

# # ============================================================================
# # GLOBAL SETTINGS
# # ============================================================================
# GLOBAL_SEED = 42
# np.random.seed(GLOBAL_SEED)

# # ============================================================================
# # DATACLASSES
# # ============================================================================
# @dataclass
# class EpisodeConf:
#     Delta: float      
#     T_slots: int      
#     seed: int

# @dataclass
# class AgentRanges:
#     lam_sec_min: float
#     lam_sec_max: float
#     f_local_min: float
#     f_local_max: float
#     m_local_min: float
#     m_local_max: float

# @dataclass
# class TaskFeatureDist:
#     b_median: float = 3.0
#     b_sigma_g: float = 1.3

#     rho_median: float = 1.2e9
#     rho_sigma_g: float = 1.2

#     mem_median: float = 64.0
#     mem_sigma_g: float = 1.1

#     p_deadline: float = 0.25
#     deadline_min: float = 0.3
#     deadline_max: float = 1.5

#     p_non_atomic: float = 0.35
#     split_ratio_min: float = 0.30
#     split_ratio_max: float = 0.80

#     modality_probs: Optional[List[float]] = None
#     modality_labels: Optional[List[str]] = None

# @dataclass
# class GlobalConfig:
#     name: str
#     N_agents: int
#     Episode: EpisodeConf
#     AgentRanges: AgentRanges
#     TaskDist: TaskFeatureDist

# # ============================================================================
# # UTILS
# # ============================================================================
# def _z_from_p(p):
#     Z = {0.90:1.2816, 0.95:1.6448, 0.975:1.9600, 0.99:2.3263}
#     return Z[min(Z.keys(), key=lambda x: abs(x - p))]

# def lognormal_from_median_sigma(rng, median, sigma_g, qcap=0.99):
#     mu = math.log(max(median, 1e-12))
#     sigma = math.log(max(sigma_g, 1.0 + 1e-6))
#     x = float(rng.lognormal(mu, sigma))
#     cap = median * math.exp(_z_from_p(qcap) * sigma)
#     return min(x, cap)

# @dataclass
# class Agent:
#     agent_id: int
#     f_local: float
#     m_local: float
#     lam_sec: float

# # ============================================================================
# # DATA GENERATION
# # ============================================================================
# def build_agents(cfg: GlobalConfig, rng):
#     agents = []
#     for i in range(cfg.N_agents):
#         agents.append(Agent(
#             agent_id=i,
#             f_local=rng.uniform(cfg.AgentRanges.f_local_min, cfg.AgentRanges.f_local_max),
#             m_local=rng.uniform(cfg.AgentRanges.m_local_min, cfg.AgentRanges.m_local_max),
#             lam_sec=rng.uniform(cfg.AgentRanges.lam_sec_min, cfg.AgentRanges.lam_sec_max)
#         ))
#     return agents


# def sample_task(cfg: GlobalConfig, rng):
#     d = cfg.TaskDist
#     b_mb = lognormal_from_median_sigma(rng, d.b_median, d.b_sigma_g)
#     rho = lognormal_from_median_sigma(rng, d.rho_median, d.rho_sigma_g)
#     mem = lognormal_from_median_sigma(rng, d.mem_median, d.mem_sigma_g)

#     c_cycles = b_mb * rho

#     has_deadline = int(rng.random() < d.p_deadline)
#     deadline_s = rng.uniform(d.deadline_min, d.deadline_max) if has_deadline else np.nan

#     non_atomic = int(rng.random() < d.p_non_atomic)
#     split_ratio = rng.uniform(d.split_ratio_min, d.split_ratio_max) if non_atomic else 0.0

#     modality = (rng.choice(d.modality_labels) if d.modality_labels
#                 else rng.choice(["image", "video", "text", "sensor"]))

#     return dict(
#         b_mb=b_mb,
#         rho=rho,
#         c_cycles=c_cycles,
#         mem_mb=mem,
#         modality=modality,
#         has_deadline=has_deadline,
#         deadline_s=deadline_s,
#         non_atomic=non_atomic,
#         split_ratio=split_ratio
#     )


# def generate_episode_for_scenario(cfg: GlobalConfig, episode_id: int, out_dir: str):
#     rng = np.random.default_rng(cfg.Episode.seed + episode_id)
#     agents = build_agents(cfg, rng)

#     T_slots = cfg.Episode.T_slots
#     Delta = cfg.Episode.Delta

#     rows_ep, rows_agents, rows_arrivals, rows_tasks = [], [], [], []

#     # Agents
#     for a in agents:
#         rows_agents.append(asdict(a))

#     task_counter = 0

#     for t in range(T_slots):

#         t_time = t * Delta
#         arrival_phase = (t < 100)

#         if arrival_phase:
#             for a in agents:
#                 lam_slot = a.lam_sec * Delta
#                 n_new = rng.poisson(lam_slot)

#                 for _ in range(n_new):
#                     feat = sample_task(cfg, rng)
#                     deadline_time = (t_time + feat["deadline_s"]) if feat["has_deadline"] else np.nan

#                     rows_arrivals.append({
#                         "scenario": cfg.name,
#                         "episode_id": episode_id,
#                         "t_slot": t,
#                         "t_time": t_time,
#                         "agent_id": a.agent_id,
#                         "task_id": task_counter
#                     })

#                     rows_tasks.append({
#                         "scenario": cfg.name,
#                         "episode_id": episode_id,
#                         "task_id": task_counter,
#                         "agent_id": a.agent_id,
#                         "t_arrival_slot": t,
#                         "t_arrival_time": t_time,
#                         "b_mb": feat["b_mb"],
#                         "rho_cyc_per_mb": feat["rho"],
#                         "c_cycles": feat["c_cycles"],
#                         "mem_mb": feat["mem_mb"],
#                         "modality": feat["modality"],
#                         "has_deadline": feat["has_deadline"],
#                         "deadline_s": feat["deadline_s"],
#                         "deadline_time": deadline_time,
#                         "non_atomic": feat["non_atomic"],
#                         "split_ratio": feat["split_ratio"]
#                     })
#                     task_counter += 1

#     rows_ep.append({
#         "scenario": cfg.name,
#         "episode_id": episode_id,
#         "Delta": Delta,
#         "T_slots": T_slots,
#         "arrival_slots": 100,
#         "drain_slots": T_slots - 100,
#         "hours": (T_slots * Delta) / 3600.0,
#         "N_agents": cfg.N_agents
#     })

#     # SAVE DATA
#     dfs = {
#         "episodes": pd.DataFrame(rows_ep),
#         "agents": pd.DataFrame(rows_agents),
#         "arrivals": pd.DataFrame(rows_arrivals),
#         "tasks": pd.DataFrame(rows_tasks)
#     }

#     os.makedirs(out_dir, exist_ok=True)
#     for name, df in dfs.items():
#         df.to_csv(os.path.join(out_dir, f"{name}.csv"), index=False)

#     return dfs


# # ============================================================================
# # SUMMARY + PLOTTING
# # ============================================================================
# def summarize_and_plot(dfs: Dict[str, pd.DataFrame], out_dir: str):
#     os.makedirs(out_dir, exist_ok=True)

#     tasks = dfs["tasks"]
#     arrivals = dfs["arrivals"]
#     episodes = dfs["episodes"]

#     def q(x, p):
#         x = x.dropna()
#         return float(np.nanquantile(x, p)) if len(x) else np.nan

#     summary = {
#         "n_tasks": len(tasks),
#         "n_arrivals": len(arrivals),
#         "tasks_per_hour": len(tasks) / episodes.iloc[0]["hours"],
#         "b_mb_median": q(tasks["b_mb"], 0.5),
#         "rho_median": q(tasks["rho_cyc_per_mb"], 0.5),
#         "c_cycles_p99": q(tasks["c_cycles"], 0.99),
#         "deadline_share": float((tasks["has_deadline"] == 1).mean()),
#         "non_atomic_share": float((tasks["non_atomic"] == 1).mean())
#     }

#     pd.DataFrame([summary]).to_csv(os.path.join(out_dir, "summary_stats.csv"), index=False)

#     # Plots
#     def hist_plot(series, title, fname):
#         plt.figure()
#         s = series.dropna()
#         plt.hist(s, bins=40) if len(s) else None
#         plt.title(title)
#         plt.tight_layout()
#         plt.savefig(os.path.join(out_dir, fname))
#         plt.close()

#     hist_plot(tasks["b_mb"], "Task Size (MB)", "hist_b_mb.png")
#     hist_plot(tasks["c_cycles"], "Total Cycles", "hist_c_cycles.png")


# # ============================================================================
# # META JSON WRITER
# # ============================================================================
# def write_metadata_json(ep_dir: str, cfg_list: List[GlobalConfig]):
#     meta = {
#         "schema_version": "2.0",
#         "generated_at": time.strftime("%Y-%m-%d %H:%M:%S"),
#         "python_version": platform.python_version(),
#         "user": getpass.getuser(),
#         "episode_folder": ep_dir,
#         "scenarios": [],
#         "notes": {
#             "slots": "100 arrival + 10 drain",
#             "structure": "datasets/ep_XXX/{light,moderate,heavy}/",
#             "usage": "Feed into MDP environment"
#         }
#     }

#     for cfg in cfg_list:
#         meta["scenarios"].append({
#             "name": cfg.name,
#             "N_agents": cfg.N_agents,
#             "Episode": asdict(cfg.Episode),
#             "AgentRanges": asdict(cfg.AgentRanges),
#             "TaskDist": asdict(cfg.TaskDist)
#         })

#     with open(os.path.join(ep_dir, "dataset_metadata.json"), "w", encoding="utf-8") as f:
#         json.dump(meta, f, indent=2, ensure_ascii=False)


# # ============================================================================
# # MAIN RUNNER
# # ============================================================================
# def generate_all_scenarios_in_episode(ep_index: int, out_root="./datasets_V02"):

#     # HOODIE-style episode setup
#     BASE_EP = EpisodeConf(Delta=1.0, T_slots=110, seed=GLOBAL_SEED + ep_index)

#     scenarios = [
#         GlobalConfig(
#             name="light",
#             N_agents=18,
#             Episode=replace(BASE_EP, seed=GLOBAL_SEED + 1000 + ep_index),
#             AgentRanges=AgentRanges(0.01, 0.05, 0.8e9, 2.4e9, 3e3, 8e3),
#             TaskDist=TaskFeatureDist(b_median=2.0, b_sigma_g=1.55)
#         ),
#         GlobalConfig(
#             name="moderate",
#             N_agents=18,
#             Episode=replace(BASE_EP, seed=GLOBAL_SEED + 2000 + ep_index),
#             AgentRanges=AgentRanges(0.05, 0.20, 0.8e9, 2.4e9, 3e3, 8e3),
#             TaskDist=TaskFeatureDist(b_median=3.0, b_sigma_g=1.60)
#         ),
#         GlobalConfig(
#             name="heavy",
#             N_agents=18,
#             Episode=replace(BASE_EP, seed=GLOBAL_SEED + 3000 + ep_index),
#             AgentRanges=AgentRanges(0.20, 0.80, 0.8e9, 2.4e9, 3e3, 8e3),
#             TaskDist=TaskFeatureDist(b_median=5.0, b_sigma_g=1.70)
#         )
#     ]

#     # Episode folder
#     ep_dir = os.path.join(out_root, f"ep_{ep_index:03d}")
#     os.makedirs(ep_dir, exist_ok=True)

#     # Generate for each scenario
#     for cfg in scenarios:
#         scen_dir = os.path.join(ep_dir, cfg.name)
#         dfs = generate_episode_for_scenario(cfg, ep_index, scen_dir)
#         summarize_and_plot(dfs, scen_dir)

#     # Write metadata JSON
#     write_metadata_json(ep_dir, scenarios)

#     return ep_dir


# # ============================================================================
# # MAIN
# # ============================================================================
# if __name__ == "__main__":
#     ep_dir = generate_all_scenarios_in_episode(ep_index=0, out_root="./datasets_V02")
#     print("Generated:", ep_dir)


Generated: ./datasets\ep_000


<h1> V01 </h1>

مقاله مرجع می‌گوید هر اپیزود شامل T = 110 تایم‌اسلات است (100 اسلات برای تصمیم‌گیری + 10 اسلات برای خالی کردن صف‌ها)، کل آموزش در 5000 اپیزود اجرا شده است. محیط‌های DRL معمولاً event-driven هستند.

updated

In [8]:
# -*- coding: utf-8 -*-
"""
Data generator for Edge–MEC–Cloud with Poisson arrivals (per-second),
lognormal task features (parameterized by median & sigma_g), and policy-agnostic outputs.

HOODIE-style timing:
  - Each episode has T = 110 time-slots:
      * T_decision = 100 slots for decision-making (with arrivals)
      * T_drain    = 10 slots for draining queues (NO new arrivals)
  - DRL environment can later consume this as event-driven using the time-stamps.

Supports THREE SCENARIOS (light / moderate / heavy) similar to HOODIE experiments.
For each episode we:
  - synthesize time-stamped arrivals and task features for ALL scenarios
  - save CSV + a per-episode dataset_metadata.json
  - plot distribution figures (PNG) for key variables
  - export a summary_stats.csv with quantiles/means per scenario

Directory layout (per episode):
  datasets/
    ep_000/
      light/
        episodes.csv
        agents.csv
        arrivals.csv
        tasks.csv
        summary_stats.csv
        *.png
      moderate/
        ...
      heavy/
        ...
      dataset_metadata.json   # meta info for ALL scenarios in this episode
"""

from __future__ import annotations
from dataclasses import dataclass, asdict, replace
from typing import List, Dict, Optional
import numpy as np
import pandas as pd
import random, math, os, json
import matplotlib.pyplot as plt
import hashlib, time, platform, getpass

# -------------------------
# reproducibility
# -------------------------
GLOBAL_SEED = 42
rng_global = np.random.default_rng(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

# -------------------------
# configuration dataclasses
# -------------------------
@dataclass
class EpisodeConf:
    Delta: float        # seconds per slot
    T_slots: int        # total number of slots in the episode (e.g. 110)
    T_decision: int     # number of slots with arrivals (e.g. 100)
    T_drain: int        # number of drain slots without arrivals (e.g. 10)
    seed: int

@dataclass
class AgentRanges:
    # arrival rate per SECOND (will be multiplied by Delta per slot)
    lam_sec_min: float
    lam_sec_max: float
    # optional local capacities (kept as meta for later)
    f_local_min: float
    f_local_max: float
    m_local_min: float
    m_local_max: float

@dataclass
class TaskFeatureDist:
    # Lognormal parameterization via median and sigma_g (geometric std).
    b_median: float = 3.0          # MB
    b_sigma_g: float = 1.5

    rho_median: float = 1.2e9      # cycles / MB
    rho_sigma_g: float = 1.5

    mem_median: float = 64.0       # MB
    mem_sigma_g: float = 1.5

    p_deadline: float = 0.25
    deadline_min: float = 0.3      # seconds (relative)
    deadline_max: float = 1.5

    p_non_atomic: float = 0.35
    split_ratio_min: float = 0.30  # fraction of task size that CAN be split
    split_ratio_max: float = 0.80

    # Optional modality probabilities (image, video, text, sensor)
    modality_probs: Optional[List[float]] = None
    modality_labels: List[str] = None

@dataclass
class GlobalConfig:
    name: str
    N_agents: int
    Episode: EpisodeConf
    AgentRanges: AgentRanges
    TaskDist: TaskFeatureDist

# -------------------------
# asserts / validation
# -------------------------
def _validate_cfg(cfg: GlobalConfig) -> None:
    assert cfg.N_agents > 0
    assert cfg.Episode.Delta > 0
    assert cfg.Episode.T_slots > 0
    assert cfg.Episode.T_decision > 0
    assert cfg.Episode.T_drain >= 0
    assert cfg.Episode.T_slots == cfg.Episode.T_decision + cfg.Episode.T_drain

    ar = cfg.AgentRanges
    assert ar.lam_sec_min >= 0 and ar.lam_sec_max >= ar.lam_sec_min
    td = cfg.TaskDist
    assert td.split_ratio_min > 0 and td.split_ratio_max <= 1.0 and td.split_ratio_max >= td.split_ratio_min
    assert td.deadline_min <= td.deadline_max
    # geometric std must be >= 1.0
    assert td.b_sigma_g >= 1.0 and td.rho_sigma_g >= 1.0 and td.mem_sigma_g >= 1.0

# -------------------------
# helpers
# -------------------------
_Z_TABLE = {
    0.90: 1.2815515655446004,
    0.95: 1.6448536269514722,
    0.975: 1.959963984540054,
    0.99: 2.3263478740408408,
    0.995: 2.5758293035489004,
    0.999: 3.090232306167813
}

def _z_from_p(p: float) -> float:
    # Use small table + nearest clamp (no SciPy dependency)
    if p in _Z_TABLE:
        return _Z_TABLE[p]
    # clamp to nearest key
    return _Z_TABLE[min(_Z_TABLE.keys(), key=lambda k: abs(k - p))]

def lognormal_quantile(median: float, sigma_g: float, p: float) -> float:
    # X ~ LogNormal(mu, sigma) with median=exp(mu), sigma_g=exp(sigma)
    # quantile(p) = median * exp( z_p * ln(sigma_g) )
    z = _z_from_p(p)
    return median * math.exp(z * math.log(max(sigma_g, 1.0 + 1e-6)))

def lognormal_from_median_sigma_g(
    rng: np.random.Generator,
    median: float,
    sigma_g: float,
    qcap: float | None = 0.99
) -> float:
    """
    Draw from LogNormal with given median and geometric std:
      X ~ LogNormal(mu, sigma) where median = exp(mu), sigma_g = exp(sigma).
      => mu = ln(median), sigma = ln(sigma_g)
    """
    mu = math.log(max(median, 1e-12))
    sigma = math.log(max(sigma_g, 1.0 + 1e-6))
    x = float(rng.lognormal(mean=mu, sigma=sigma))
    if qcap is not None:
        cap = lognormal_quantile(median, sigma_g, qcap)
        x = min(x, cap)
    return x

# -------------------------
# entities
# -------------------------
@dataclass
class Agent:
    agent_id: int
    f_local: float
    m_local: float
    lam_sec: float   # Poisson rate per second (not per-slot)

def build_agents(cfg: GlobalConfig, rng: np.random.Generator) -> List[Agent]:
    agents: List[Agent] = []
    for i in range(cfg.N_agents):
        lam_sec = rng.uniform(cfg.AgentRanges.lam_sec_min, cfg.AgentRanges.lam_sec_max)
        f_loc   = rng.uniform(cfg.AgentRanges.f_local_min, cfg.AgentRanges.f_local_max)
        m_loc   = rng.uniform(cfg.AgentRanges.m_local_min, cfg.AgentRanges.m_local_max)
        agents.append(Agent(agent_id=i, f_local=f_loc, m_local=m_loc, lam_sec=lam_sec))
    return agents

# -------------------------
# task features
# -------------------------
def _modality_choice(rng: np.random.Generator, d: TaskFeatureDist) -> str:
    labels = d.modality_labels or ["image", "video", "text", "sensor"]
    if d.modality_probs is None:
        probs = [0.3, 0.2, 0.3, 0.2]
    else:
        probs = d.modality_probs
        assert abs(sum(probs) - 1.0) < 1e-6 and len(probs) == len(labels)
    return rng.choice(labels, p=probs)

def sample_task_features(cfg: GlobalConfig, rng: np.random.Generator, qcap: float = 0.99) -> Dict[str, float]:
    d = cfg.TaskDist
    b_mb   = lognormal_from_median_sigma_g(rng, d.b_median,   d.b_sigma_g,   qcap=qcap)
    rho    = lognormal_from_median_sigma_g(rng, d.rho_median, d.rho_sigma_g, qcap=qcap)
    c      = b_mb * rho                              # total cycles
    mem_mb = lognormal_from_median_sigma_g(rng, d.mem_median, d.mem_sigma_g, qcap=qcap)
    modality = _modality_choice(rng, d)
    has_deadline = int(rng.random() < d.p_deadline)
    deadline_s   = np.nan
    if has_deadline:
        deadline_s = float(rng.uniform(d.deadline_min, d.deadline_max))
    non_atomic = int(rng.random() < d.p_non_atomic)
    split_ratio = float(rng.uniform(d.split_ratio_min, d.split_ratio_max)) if non_atomic else 0.0
    
    return dict(
        b_mb=b_mb, rho=rho, c_cycles=c, mem_mb=mem_mb, modality=modality,
        has_deadline=has_deadline, deadline_s=deadline_s, non_atomic=non_atomic, split_ratio=split_ratio
    )

# -------------------------
# episode generator (arrivals only)
# -------------------------
def run_episode(
    cfg: GlobalConfig,
    agents: List[Agent],
    episode_id: int = 0,
    qcap: float = 0.99
) -> Dict[str, pd.DataFrame]:
    """
    Generate time-stamped arrivals and task features for ONE episode of ONE scenario.

    HOODIE-style:
      - t = 0 .. T_decision-1 → arrivals via Poisson
      - t = T_decision .. T_slots-1 → NO new arrivals (drain phase)
    """
    _validate_cfg(cfg)
    rng_local = np.random.default_rng(cfg.Episode.seed + episode_id)

    rows_episodes: List[Dict] = []
    rows_agents:   List[Dict] = []
    rows_arrivals: List[Dict] = []
    rows_tasks:    List[Dict] = []

    Delta      = cfg.Episode.Delta
    T_slots    = cfg.Episode.T_slots
    T_decision = cfg.Episode.T_decision

    # agents (include scenario for easier joins)
    for a in agents:
        rows_agents.append({
            "scenario": cfg.name,
            "agent_id": a.agent_id,
            "f_local": a.f_local,
            "m_local": a.m_local,
            "lam_sec": a.lam_sec
        })

    task_id_counter = 0

    for t in range(T_slots):
        t_time = t * Delta

        # Only first T_decision slots have new arrivals (HOODIE style)
        if t >= T_decision:
            continue

        for a in agents:
            # per-slot rate from per-second rate:
            lam_slot = a.lam_sec * Delta
            n_new = rng_local.poisson(lam=lam_slot)
            if n_new <= 0:
                continue
            for _ in range(n_new):
                feat = sample_task_features(cfg, rng_local, qcap=qcap)

                # absolute deadline time (NaN if none)
                if np.isnan(feat["deadline_s"]):
                    deadline_time = np.nan
                else:
                    deadline_time = t_time + feat["deadline_s"]

                action_space_hint = "continuous" if feat["non_atomic"] == 1 else "discrete"

                rows_arrivals.append({
                    "scenario": cfg.name,
                    "episode_id": episode_id,
                    "t_slot": t,
                    "t_time": t_time,
                    "agent_id": a.agent_id,
                    "task_id": task_id_counter
                })

                rows_tasks.append({
                    "scenario": cfg.name,
                    "episode_id": episode_id,
                    "task_id": task_id_counter,
                    "agent_id": a.agent_id,
                    "t_arrival_slot": t,
                    "t_arrival_time": t_time,
                    "b_mb": feat["b_mb"],
                    "rho_cyc_per_mb": feat["rho"],
                    "c_cycles": feat["c_cycles"],
                    "mem_mb": feat["mem_mb"],
                    "modality": feat["modality"],
                    "has_deadline": feat["has_deadline"],
                    "deadline_s": feat["deadline_s"],
                    "deadline_time": deadline_time,
                    "non_atomic": feat["non_atomic"],
                    "split_ratio": feat["split_ratio"],
                    "action_space_hint": action_space_hint
                })

                task_id_counter += 1

    rows_episodes.append({
        "scenario": cfg.name,
        "episode_id": episode_id,
        "Delta": Delta,
        "T_slots": T_slots,
        "T_decision": T_decision,
        "T_drain": cfg.Episode.T_drain,
        "hours": T_slots * Delta / 3600.0,
        "N_agents": len(agents),
        "seed": cfg.Episode.seed + episode_id
    })

    episodes_df = pd.DataFrame(rows_episodes)
    agents_df   = pd.DataFrame(rows_agents)
    arrivals_df = pd.DataFrame(rows_arrivals)
    tasks_df    = pd.DataFrame(rows_tasks)

    # Optimize dtypes
    if len(tasks_df):
        tasks_df["modality"] = tasks_df["modality"].astype("category")
        tasks_df["action_space_hint"] = tasks_df["action_space_hint"].astype("category")
        # ints
        for col in ["episode_id", "task_id", "agent_id", "t_arrival_slot", "has_deadline", "non_atomic"]:
            if col in tasks_df:
                tasks_df[col] = tasks_df[col].astype("int32")
        # floats
        for col in ["t_arrival_time", "b_mb", "rho_cyc_per_mb", "c_cycles", "mem_mb",
                    "deadline_s", "deadline_time", "split_ratio"]:
            if col in tasks_df:
                tasks_df[col] = tasks_df[col].astype("float32")

    if len(arrivals_df):
        for col in ["episode_id", "t_slot", "agent_id", "task_id"]:
            if col in arrivals_df:
                arrivals_df[col] = arrivals_df[col].astype("int32")
        for col in ["t_time"]:
            if col in arrivals_df:
                arrivals_df[col] = arrivals_df[col].astype("float32")

    if len(agents_df):
        for col in ["agent_id"]:
            agents_df[col] = agents_df[col].astype("int32")
        for col in ["f_local", "m_local", "lam_sec"]:
            agents_df[col] = agents_df[col].astype("float64")

    if len(episodes_df):
        for col in ["episode_id", "N_agents", "T_slots", "T_decision", "T_drain"]:
            if col in episodes_df:
                episodes_df[col] = episodes_df[col].astype("int32")
        for col in ["Delta", "hours"]:
            if col in episodes_df:
                episodes_df[col] = episodes_df[col].astype("float32")

    return {
        "episodes": episodes_df,
        "agents":   agents_df,
        "arrivals": arrivals_df,
        "tasks":    tasks_df
    }

# -------------------------
# save & plotting utilities
# -------------------------
def save_dataset(dfs: Dict[str, pd.DataFrame], out_dir: str = ".") -> Dict[str, str]:
    os.makedirs(out_dir, exist_ok=True)
    paths: Dict[str, str] = {}
    for name, df in dfs.items():
        csv_path = os.path.join(out_dir, f"{name}.csv")
        df.to_csv(csv_path, index=False)
        paths[name + "_csv"] = csv_path
    return paths

def _config_fingerprint(cfg: GlobalConfig) -> str:
    s = json.dumps({
        "scenario": cfg.name,
        "Episode": asdict(cfg.Episode),
        "AgentRanges": asdict(cfg.AgentRanges),
        "TaskDist": asdict(cfg.TaskDist)
    }, sort_keys=True).encode("utf-8")
    return hashlib.sha256(s).hexdigest()[:16]

def save_episode_meta(
    cfgs: List[GlobalConfig],
    ep_dir: str,
    qcap: float = 0.99
) -> str:
    """
    Save a single dataset_metadata.json inside ep_xxx containing
    config info for ALL scenarios used in this episode.
    """
    meta = {
        "schema_version": "1.0.0",
        "generated_at_utc": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
        "env": {
            "python": platform.python_version(),
            "user": getpass.getuser()
        },
        "episodes_root": os.path.abspath(ep_dir),
        "notes": {
            "policy_agnostic": True,
            "queueing": "not simulated here",
            "timing": {
                "description": "HOODIE-style: T_decision slots with arrivals + T_drain slots without arrivals.",
            },
            "clipping": {
                "enabled": True,
                "method": "lognormal analytic quantile cap",
                "qcap": qcap,
                "z_table_keys": sorted(list(_Z_TABLE.keys()))
            },
            "action_space_hint": "derived from non_atomic; final decision belongs to environment"
        },
        "scenarios": []
    }

    for cfg in cfgs:
        meta["scenarios"].append({
            "name": cfg.name,
            "fingerprint": _config_fingerprint(cfg),
            "Episode": asdict(cfg.Episode),
            "N_agents": cfg.N_agents,
            "AgentRanges": asdict(cfg.AgentRanges),
            "TaskDist": asdict(cfg.TaskDist),
            "units": {
                "Delta": "seconds",
                "lam_sec": "tasks per second (per agent)",
                "per_slot_rate": "lam_sec * Delta",
                "b_mb": "MB",
                "rho_cyc_per_mb": "CPU cycles per MB",
                "c_cycles": "CPU cycles",
                "mem_mb": "MB",
                "deadline_s": "seconds (relative); deadline_time = t_arrival_time + deadline_s",
                "f_local": "Hz",
                "m_local": "MB"
            }
        })

    path = os.path.join(ep_dir, "dataset_metadata.json")
    with open(path, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)
    return path

def summarize_and_plot(dfs: Dict[str, pd.DataFrame], out_dir: str) -> None:
    os.makedirs(out_dir, exist_ok=True)
    tasks = dfs["tasks"].copy()
    arrivals = dfs["arrivals"].copy()
    episodes = dfs["episodes"].copy()

    # ---- summary stats
    def q(series: pd.Series, p: float):
        s = series.dropna()
        return float(np.nanquantile(s, p)) if len(s) else float("nan")

    tasks_per_hour = float(len(tasks)) / float(episodes.iloc[0]["hours"]) if len(episodes) and episodes.iloc[0]["hours"] > 0 else float("nan")

    # modality distribution
    if len(tasks):
        mod_counts = tasks["modality"].value_counts(dropna=False)
        mod_dist = {str(k): int(v) for k, v in mod_counts.to_dict().items()}
    else:
        mod_dist = {}

    summary = {
        "n_tasks": [len(tasks)],
        "n_arrivals": [len(arrivals)],
        "tasks_per_hour": [tasks_per_hour],
        "b_mb_median": [q(tasks["b_mb"], 0.5)] if len(tasks) else [float("nan")],
        "b_mb_p90": [q(tasks["b_mb"], 0.9)] if len(tasks) else [float("nan")],
        "b_mb_p99": [q(tasks["b_mb"], 0.99)] if len(tasks) else [float("nan")],
        "rho_median": [q(tasks["rho_cyc_per_mb"], 0.5)] if len(tasks) else [float("nan")],
        "rho_p90": [q(tasks["rho_cyc_per_mb"], 0.9)] if len(tasks) else [float("nan")],
        "c_cycles_median": [q(tasks["c_cycles"], 0.5)] if len(tasks) else [float("nan")],
        "c_cycles_p90": [q(tasks["c_cycles"], 0.9)] if len(tasks) else [float("nan")],
        "c_cycles_p99": [q(tasks["c_cycles"], 0.99)] if len(tasks) else [float("nan")],
        "deadline_share": [float((tasks["has_deadline"] == 1).mean()) if len(tasks) else float("nan")],
        "non_atomic_share": [float((tasks["non_atomic"] == 1).mean()) if len(tasks) else float("nan")],
        "modality_counts_json": [json.dumps(mod_dist)]
    }
    pd.DataFrame(summary).to_csv(os.path.join(out_dir, "summary_stats.csv"), index=False)

    # ---- plots (each in its own figure)
    def hist_plot(series: pd.Series, title: str, fname: str, logx: bool = False):
        plt.figure()
        s = series.dropna()
        if len(s) == 0:
            plt.title(title + " (no data)")
        else:
            plt.hist(s, bins=50)
            if logx:
                plt.xscale("log")
            plt.title(title)
            plt.xlabel(title)
            plt.ylabel("count")
        plt.tight_layout()
        plt.savefig(os.path.join(out_dir, fname), dpi=160)
        plt.close()

    if len(tasks):
        hist_plot(tasks["b_mb"],            title="Task size (MB)",                fname="hist_b_mb.png", logx=True)
        hist_plot(tasks["rho_cyc_per_mb"],  title="Compute density (cycles/MB)",   fname="hist_rho.png",  logx=True)
        hist_plot(tasks["c_cycles"],        title="Total cycles",                  fname="hist_c_cycles.png", logx=True)
        hist_plot(tasks["deadline_s"],      title="Deadline (s)",                  fname="hist_deadline_s.png", logx=False)
        hist_plot(tasks.loc[tasks["non_atomic"] == 1, "split_ratio"],
                  title="Split ratio (only non-atomic)", fname="hist_split_ratio.png", logx=False)

    # arrivals per agent
    if len(arrivals):
        per_agent = arrivals.groupby("agent_id").size()
        plt.figure()
        plt.bar(per_agent.index.astype(str), per_agent.values)
        plt.title("Arrivals per agent")
        plt.xlabel("agent_id")
        plt.ylabel("count")
        plt.tight_layout()
        plt.savefig(os.path.join(out_dir, "bar_arrivals_per_agent.png"), dpi=160)
        plt.close()

# -------------------------
# scenario presets (light / moderate / heavy) – HOODIE-style timing
# -------------------------
DEFAULT_DELTA = 1.0      # 1 second per slot (unit choice)
T_DECISION = 100         # 100 decision slots
T_DRAIN    = 10          # 10 drain slots
DEFAULT_T_SLOTS = T_DECISION + T_DRAIN  # 110 total

BASE_EPISODE = EpisodeConf(
    Delta=DEFAULT_DELTA,
    T_slots=DEFAULT_T_SLOTS,
    T_decision=T_DECISION,
    T_drain=T_DRAIN,
    seed=GLOBAL_SEED
)

BASE_AGENT_RANGES = AgentRanges(
    lam_sec_min=0.02, lam_sec_max=0.80,
    f_local_min=0.8e9, f_local_max=2.4e9,
    m_local_min=3e3,  m_local_max=8e3  # MB
)
BASE_TASK_DIST = TaskFeatureDist()

SCENARIOS: List[GlobalConfig] = [
    GlobalConfig(
        name="light",
        N_agents=18,
        Episode=replace(BASE_EPISODE, seed=GLOBAL_SEED + 101),
        AgentRanges=replace(BASE_AGENT_RANGES, lam_sec_min=0.01, lam_sec_max=0.05),
        TaskDist=replace(
            BASE_TASK_DIST,
            b_median=2.0,  b_sigma_g=1.55,
            rho_median=1.0e9, rho_sigma_g=1.45,
            mem_median=64.0, mem_sigma_g=1.40,
            p_deadline=0.15, deadline_min=0.8, deadline_max=2.0,
            p_non_atomic=0.25, split_ratio_min=0.25, split_ratio_max=0.75
        )
    ),
    GlobalConfig(
        name="moderate",
        N_agents=18,
        Episode=replace(BASE_EPISODE, seed=GLOBAL_SEED + 202),
        AgentRanges=replace(BASE_AGENT_RANGES, lam_sec_min=0.05, lam_sec_max=0.20),
        TaskDist=replace(
            BASE_TASK_DIST,
            b_median=3.0,  b_sigma_g=1.60,
            rho_median=1.2e9, rho_sigma_g=1.50,
            mem_median=64.0, mem_sigma_g=1.45,
            p_deadline=0.25, deadline_min=0.5, deadline_max=1.5,
            p_non_atomic=0.35, split_ratio_min=0.30, split_ratio_max=0.80
        )
    ),
    GlobalConfig(
        name="heavy",
        N_agents=18,
        Episode=replace(BASE_EPISODE, seed=GLOBAL_SEED + 303),
        AgentRanges=replace(BASE_AGENT_RANGES, lam_sec_min=0.20, lam_sec_max=0.80),
        TaskDist=replace(
            BASE_TASK_DIST,
            b_median=5.0,  b_sigma_g=1.70,
            rho_median=1.5e9, rho_sigma_g=1.55,
            mem_median=64.0, mem_sigma_g=1.50,
            p_deadline=0.35, deadline_min=0.3, deadline_max=1.0,
            p_non_atomic=0.45, split_ratio_min=0.40, split_ratio_max=0.85
        )
    )
]

# -------------------------
# drivers: per-scenario & all-scenarios
# -------------------------
def main_generate_for_scenario(
    cfg: GlobalConfig,
    agents: List[Agent],
    episode_id: int,
    ep_dir: str,
    qcap: float = 0.99
) -> Dict[str, str]:
    """
    Generate ONE episode for ONE scenario under:
        ep_dir/<scenario>/
    For example: datasets/ep_000/heavy/
    """
    _validate_cfg(cfg)

    scenario_dir = os.path.join(ep_dir, cfg.name)
    os.makedirs(scenario_dir, exist_ok=True)

    dfs = run_episode(cfg, agents, episode_id=episode_id, qcap=qcap)
    paths = save_dataset(dfs, out_dir=scenario_dir)
    summarize_and_plot(dfs, out_dir=scenario_dir)

    return paths

def generate_all_scenarios(
    episodes_each: int = 1,
    out_root: str = "./datasets",
    qcap: float = 0.99
) -> Dict[str, Dict[str, str]]:
    """
    Layout:
        out_root/ep_000/<scenario>/
        out_root/ep_001/<scenario>/
        ...
    And inside each ep_xxx we store dataset_metadata.json
    with info for ALL scenarios.

    NOTE:
      - To fully mimic HOODIE training (5000 episodes), call:
            generate_all_scenarios(episodes_each=5000, ...)
      - Default episodes_each=1 is for debugging.
    """
    results: Dict[str, Dict[str, str]] = {}

    # For stability, build agents per scenario once (same pool across episodes for that scenario)
    agents_per_scenario: Dict[str, List[Agent]] = {}
    for cfg in SCENARIOS:
        rng_agents = np.random.default_rng(cfg.Episode.seed + 10_000)
        agents_per_scenario[cfg.name] = build_agents(cfg, rng_agents)

    for ep in range(episodes_each):
        ep_name = f"ep_{ep:03d}"
        ep_dir = os.path.join(out_root, ep_name)
        os.makedirs(ep_dir, exist_ok=True)

        episode_paths: Dict[str, str] = {}
        for cfg in SCENARIOS:
            agents = agents_per_scenario[cfg.name]
            paths = main_generate_for_scenario(
                cfg=cfg,
                agents=agents,
                episode_id=ep,
                ep_dir=ep_dir,
                qcap=qcap
            )
            # keys like: heavy_episodes_csv, heavy_tasks_csv, ...
            for k, v in paths.items():
                episode_paths[f"{cfg.name}_{k}"] = v

        # per-episode metadata (ALL scenarios)
        meta_path = save_episode_meta(SCENARIOS, ep_dir=ep_dir, qcap=qcap)
        episode_paths["metadata_json"] = meta_path

        results[ep_name] = episode_paths

    return results

# -------------------------
# sanity checks
# -------------------------
def sanity_check_episode(ep_dir: str, scenario_names: List[str]) -> None:
    """
    Lightweight sanity checks for one ep_xxx:
      - basic file presence
      - tasks vs arrivals count & task_id consistency
      - agent_id consistency
      - basic value ranges (positivity, deadlines, split_ratio)
      - rough consistency between lambda and realized tasks/hour (on decision slots)
    """
    print(f"[sanity] Checking episode directory: {ep_dir}")
    meta_path = os.path.join(ep_dir, "dataset_metadata.json")
    if not os.path.isfile(meta_path):
        print(f"  [WARN] No dataset_metadata.json found in {ep_dir}")
    else:
        try:
            with open(meta_path, "r", encoding="utf-8") as f:
                meta = json.load(f)
            print(f"  [OK] Loaded metadata with {len(meta.get('scenarios', []))} scenarios")
        except Exception as e:
            print(f"  [WARN] Failed to read metadata: {e}")

    for scen in scenario_names:
        scen_dir = os.path.join(ep_dir, scen)
        if not os.path.isdir(scen_dir):
            print(f"  [WARN] Scenario dir missing: {scen_dir}")
            continue

        try:
            episodes_df = pd.read_csv(os.path.join(scen_dir, "episodes.csv"))
            agents_df   = pd.read_csv(os.path.join(scen_dir, "agents.csv"))
            arrivals_df = pd.read_csv(os.path.join(scen_dir, "arrivals.csv"))
            tasks_df    = pd.read_csv(os.path.join(scen_dir, "tasks.csv"))
        except Exception as e:
            print(f"  [WARN] Failed to load CSVs for scenario '{scen}': {e}")
            continue

        print(f"  [scenario={scen}] n_tasks={len(tasks_df)}, n_arrivals={len(arrivals_df)}, n_agents={len(agents_df)}")

        # 1) non-empty checks
        if len(tasks_df) == 0 or len(arrivals_df) == 0:
            print(f"    [WARN] Empty tasks or arrivals for scenario '{scen}'")
            continue

        # 2) tasks vs arrivals counts & unique task_ids
        if len(tasks_df) != len(arrivals_df):
            print(f"    [WARN] tasks ({len(tasks_df)}) != arrivals ({len(arrivals_df)})")

        if tasks_df["task_id"].nunique() != len(tasks_df):
            print(f"    [WARN] Duplicate task_id in tasks.csv for scenario '{scen}'")

        # 3) agent_id consistency
        agents_set = set(agents_df["agent_id"].tolist())
        arr_agents = set(arrivals_df["agent_id"].tolist())
        task_agents = set(tasks_df["agent_id"].tolist())
        if not arr_agents.issubset(agents_set):
            print(f"    [WARN] Some arrival agent_ids not in agents table for '{scen}'")
        if not task_agents.issubset(agents_set):
            print(f"    [WARN] Some task agent_ids not in agents table for '{scen}'")

        # 4) basic value ranges
        if (tasks_df["b_mb"] <= 0).any():
            print(f"    [WARN] Non-positive b_mb values in tasks for '{scen}'")
        if (tasks_df["mem_mb"] <= 0).any():
            print(f"    [WARN] Non-positive mem_mb values in tasks for '{scen}'")
        if (tasks_df["c_cycles"] <= 0).any():
            print(f"    [WARN] Non-positive c_cycles values in tasks for '{scen}'")

        # deadlines: deadline_time >= t_arrival_time when has_deadline
        with_deadline = tasks_df["has_deadline"] == 1
        if with_deadline.any():
            bad_deadlines = (tasks_df.loc[with_deadline, "deadline_time"] <
                             tasks_df.loc[with_deadline, "t_arrival_time"]).sum()
            if bad_deadlines > 0:
                print(f"    [WARN] {bad_deadlines} rows with deadline_time < t_arrival_time in '{scen}'")

        # split ratio range
        if ((tasks_df["split_ratio"] < 0) | (tasks_df["split_ratio"] > 1)).any():
            print(f"    [WARN] split_ratio out of [0,1] range in '{scen}'")
        # non_atomic=0 => split_ratio==0
        non_atomic_zero = tasks_df["non_atomic"] == 0
        if (tasks_df.loc[non_atomic_zero, "split_ratio"] != 0).any():
            print(f"    [WARN] non_atomic==0 but split_ratio != 0 in '{scen}'")

        # 5) rough lambda consistency check (using decision horizon length)
        if len(episodes_df):
            hours = float(episodes_df.iloc[0]["T_decision"] * episodes_df.iloc[0]["Delta"]) / 3600.0
            if hours > 0:
                # arrivals only exist in decision slots by construction
                tasks_per_agent = arrivals_df.groupby("agent_id").size() / hours
                merged = pd.merge(
                    agents_df[["agent_id", "lam_sec"]],
                    tasks_per_agent.rename("tasks_per_hour").reset_index(),
                    on="agent_id",
                    how="left"
                )
                merged["lam_per_hour"] = merged["lam_sec"] * 3600.0
                merged["ratio"] = merged["tasks_per_hour"] / merged["lam_per_hour"]
                too_low = (merged["ratio"] < 0.4).sum()
                too_high = (merged["ratio"] > 1.6).sum()
                if too_low + too_high > 0:
                    print(f"    [INFO] Poisson check: some agents have realized rate far from expected "
                          f"(too_low={too_low}, too_high={too_high}) in '{scen}'")

def sanity_check_root(out_root: str, scenario_names: List[str]) -> None:
    """
    Run sanity checks over all ep_* folders under out_root.
    """
    if not os.path.isdir(out_root):
        print(f"[sanity] Root directory '{out_root}' does not exist.")
        return

    episodes = sorted([d for d in os.listdir(out_root) if d.startswith("ep_")])
    if not episodes:
        print(f"[sanity] No ep_* folders found under '{out_root}'.")
        return

    for ep_name in episodes:
        ep_dir = os.path.join(out_root, ep_name)
        if os.path.isdir(ep_dir):
            sanity_check_episode(ep_dir, scenario_names)

# -------------------------
# main
# -------------------------
if __name__ == "__main__":
    out_root = "./datasets"
    # For testing: 1 episode; to get closer to the original HOODIE:
    #   generate_all_scenarios(episodes_each=5000, ...)
    results = generate_all_scenarios(episodes_each=1, out_root=out_root, qcap=0.99)
    print(json.dumps(results, indent=2))

    # basic sanity checks
    scenario_names = [cfg.name for cfg in SCENARIOS]
    sanity_check_root(out_root, scenario_names=scenario_names)


{
  "ep_000": {
    "light_episodes_csv": "./datasets\\ep_000\\light\\episodes.csv",
    "light_agents_csv": "./datasets\\ep_000\\light\\agents.csv",
    "light_arrivals_csv": "./datasets\\ep_000\\light\\arrivals.csv",
    "light_tasks_csv": "./datasets\\ep_000\\light\\tasks.csv",
    "moderate_episodes_csv": "./datasets\\ep_000\\moderate\\episodes.csv",
    "moderate_agents_csv": "./datasets\\ep_000\\moderate\\agents.csv",
    "moderate_arrivals_csv": "./datasets\\ep_000\\moderate\\arrivals.csv",
    "moderate_tasks_csv": "./datasets\\ep_000\\moderate\\tasks.csv",
    "heavy_episodes_csv": "./datasets\\ep_000\\heavy\\episodes.csv",
    "heavy_agents_csv": "./datasets\\ep_000\\heavy\\agents.csv",
    "heavy_arrivals_csv": "./datasets\\ep_000\\heavy\\arrivals.csv",
    "heavy_tasks_csv": "./datasets\\ep_000\\heavy\\tasks.csv",
    "metadata_json": "./datasets\\ep_000\\dataset_metadata.json"
  }
}
[sanity] Checking episode directory: ./datasets\ep_000
  [OK] Loaded metadata with 3 scenar

initial version

In [5]:
# # -*- coding: utf-8 -*-
# """
# Data generator for Edge–MEC–Cloud with Poisson arrivals (per-second),
# lognormal task features (parameterized by median & sigma_g), and policy-agnostic outputs.

# Now supports THREE SCENARIOS (light / moderate / heavy) similar to HOODIE experiments.
# For each scenario we:
#   - synthesize time-stamped arrivals and task features for one or more episodes
#   - save CSV + a dataset_meta.json
#   - plot distribution figures (PNG) for key variables
#   - export a summary_stats.csv with quantiles/means

# Fixes/Improvements:
# - Per-episode subfolders (avoid overwrite) → ./datasets/<scenario>/ep_XXX/
# - Flexible lognormal quantile with small z-table + clamp
# - Parameter asserts & unit consistency
# - Richer summary (p90/p99, modality dist, tasks/hour)
# - Optional modality probabilities in TaskFeatureDist
# - Dtype optimization for large datasets
# """

# from __future__ import annotations
# from dataclasses import dataclass, asdict, replace
# from typing import List, Dict, Tuple, Optional
# import numpy as np
# import pandas as pd
# import random, math, os, json
# import matplotlib.pyplot as plt
# import hashlib, time, platform, getpass

# # -------------------------
# # reproducibility
# # -------------------------
# GLOBAL_SEED = 42
# rng_global = np.random.default_rng(GLOBAL_SEED)
# random.seed(GLOBAL_SEED)
# np.random.seed(GLOBAL_SEED)

# # -------------------------
# # configuration dataclasses
# # -------------------------
# @dataclass
# class EpisodeConf:
#     Delta: float      # seconds per slot
#     T_slots: int      # number of slots in the episode
#     seed: int

# @dataclass
# class AgentRanges:
#     # arrival rate per SECOND (will be multiplied by Delta per slot)
#     lam_sec_min: float
#     lam_sec_max: float
#     # optional local capacities (kept as meta for later)
#     f_local_min: float
#     f_local_max: float
#     m_local_min: float
#     m_local_max: float

# @dataclass
# class TaskFeatureDist:
#     # Lognormal parameterization via median and sigma_g (geometric std).
#     b_median: float = 3.0          # MB
#     b_sigma_g: float = 0.6

#     rho_median: float = 1.2e9      # cycles / MB
#     rho_sigma_g: float = 0.5

#     mem_median: float = 64.0       # MB
#     mem_sigma_g: float = 0.5

#     p_deadline: float = 0.25
#     deadline_min: float = 0.3      # seconds (relative)
#     deadline_max: float = 1.5

#     p_non_atomic: float = 0.35
#     split_ratio_min: float = 0.30  # fraction of task size that CAN be split
#     split_ratio_max: float = 0.80

#     # Optional modality probabilities (image, video, text, sensor)
#     modality_probs: Optional[List[float]] = None
#     modality_labels: List[str] = None

# @dataclass
# class GlobalConfig:
#     name: str
#     N_agents: int
#     Episode: EpisodeConf
#     AgentRanges: AgentRanges
#     TaskDist: TaskFeatureDist

# # -------------------------
# # asserts / validation
# # -------------------------
# def _validate_cfg(cfg: GlobalConfig) -> None:
#     assert cfg.N_agents > 0
#     assert cfg.Episode.Delta > 0 and cfg.Episode.T_slots > 0
#     ar = cfg.AgentRanges
#     assert ar.lam_sec_min >= 0 and ar.lam_sec_max >= ar.lam_sec_min
#     td = cfg.TaskDist
#     assert td.split_ratio_min > 0 and td.split_ratio_max <= 1.0 and td.split_ratio_max >= td.split_ratio_min
#     assert td.deadline_min <= td.deadline_max
#     # geometric std must be >= 1.0 (clamped later, but assert for awareness)
#     assert td.b_sigma_g >= 1.0 and td.rho_sigma_g >= 1.0 and td.mem_sigma_g >= 1.0

# # -------------------------
# # helpers
# # -------------------------
# _Z_TABLE = {
#     0.90: 1.2815515655446004,
#     0.95: 1.6448536269514722,
#     0.975: 1.959963984540054,
#     0.99: 2.3263478740408408,
#     0.995: 2.5758293035489004,
#     0.999: 3.090232306167813
# }

# def _z_from_p(p: float) -> float:
#     # Use small table + nearest clamp (no SciPy dependency)
#     if p in _Z_TABLE: return _Z_TABLE[p]
#     # clamp to nearest key
#     return _Z_TABLE[min(_Z_TABLE.keys(), key=lambda k: abs(k - p))]

# def lognormal_quantile(median: float, sigma_g: float, p: float) -> float:
#     # X ~ LogNormal(mu, sigma) with median=exp(mu), sigma_g=exp(sigma)
#     # quantile(p) = median * exp( z_p * ln(sigma_g) )
#     z = _z_from_p(p)
#     return median * math.exp(z * math.log(max(sigma_g, 1.0 + 1e-6)))

# def lognormal_from_median_sigma_g(rng, median: float, sigma_g: float, qcap: float | None = 0.99) -> float:
#     """
#     Draw from LogNormal with given median and geometric std:
#       X ~ LogNormal(mu, sigma) where median = exp(mu), sigma_g = exp(sigma).
#       => mu = ln(median), sigma = ln(sigma_g)
#     """
#     mu = math.log(max(median, 1e-12))
#     sigma = math.log(max(sigma_g, 1.0 + 1e-6))
#     x = float(rng.lognormal(mean=mu, sigma=sigma))
#     if qcap is not None:
#         cap = lognormal_quantile(median, sigma_g, qcap)
#         x = min(x, cap)
#     return x

# # -------------------------
# # entities
# # -------------------------
# @dataclass
# class Agent:
#     agent_id: int
#     f_local: float
#     m_local: float
#     lam_sec: float   # Poisson rate per second (not per-slot)

# def build_agents(cfg: GlobalConfig, rng: np.random.Generator) -> List[Agent]:
#     agents: List[Agent] = []
#     for i in range(cfg.N_agents):
#         lam_sec = rng.uniform(cfg.AgentRanges.lam_sec_min, cfg.AgentRanges.lam_sec_max)
#         f_loc   = rng.uniform(cfg.AgentRanges.f_local_min, cfg.AgentRanges.f_local_max)
#         m_loc   = rng.uniform(cfg.AgentRanges.m_local_min, cfg.AgentRanges.m_local_max)
#         agents.append(Agent(agent_id=i, f_local=f_loc, m_local=m_loc, lam_sec=lam_sec))
#     return agents

# # -------------------------
# # task features
# # -------------------------
# def _modality_choice(rng: np.random.Generator, d: TaskFeatureDist) -> str:
#     labels = d.modality_labels or ["image","video","text","sensor"]
#     if d.modality_probs is None:
#         probs = [0.3, 0.2, 0.3, 0.2]
#     else:
#         probs = d.modality_probs
#         assert abs(sum(probs) - 1.0) < 1e-6 and len(probs) == len(labels)
#     return rng.choice(labels, p=probs)

# def sample_task_features(cfg: GlobalConfig, rng: np.random.Generator, qcap: float = 0.99) -> Dict[str, float]:
#     d = cfg.TaskDist
#     b_mb   = lognormal_from_median_sigma_g(rng, d.b_median,   d.b_sigma_g,   qcap=qcap)
#     rho    = lognormal_from_median_sigma_g(rng, d.rho_median, d.rho_sigma_g, qcap=qcap)
#     c      = b_mb * rho                              # total cycles
#     mem_mb = lognormal_from_median_sigma_g(rng, d.mem_median, d.mem_sigma_g, qcap=qcap)
#     modality = _modality_choice(rng, d)
#     has_deadline = int(rng.random() < d.p_deadline)
#     deadline_s   = np.nan
#     if has_deadline:
#         deadline_s = float(rng.uniform(d.deadline_min, d.deadline_max))
#     non_atomic = int(rng.random() < d.p_non_atomic)
#     split_ratio = float(rng.uniform(d.split_ratio_min, d.split_ratio_max)) if non_atomic else 0.0
    
#     return dict(
#         b_mb=b_mb, rho=rho, c_cycles=c, mem_mb=mem_mb, modality=modality,
#         has_deadline=has_deadline, deadline_s=deadline_s, non_atomic=non_atomic, split_ratio=split_ratio
#     )

# # -------------------------
# # episode generator (arrivals only)
# # -------------------------
# def run_episode(cfg: GlobalConfig, agents: List[Agent], episode_id: int = 0, qcap: float = 0.99) -> Dict[str, pd.DataFrame]:
#     """
#     Generate time-stamped arrivals and task features for one episode.
#     """
#     _validate_cfg(cfg)
#     rng_local = np.random.default_rng(cfg.Episode.seed + episode_id)

#     rows_episodes: List[Dict] = []
#     rows_agents:   List[Dict] = [asdict(a) for a in agents]
#     rows_arrivals: List[Dict] = []
#     rows_tasks:    List[Dict] = []

#     Delta   = cfg.Episode.Delta
#     T_slots = cfg.Episode.T_slots

#     task_id_counter = 0

#     for t in range(T_slots):
#         t_time = t * Delta
#         for a in agents:
#             # per-slot rate from per-second rate:
#             lam_slot = a.lam_sec * Delta
#             n_new = rng_local.poisson(lam=lam_slot)
#             if n_new <= 0:
#                 continue
#             for _ in range(n_new):
#                 feat = sample_task_features(cfg, rng_local, qcap=qcap)

#                 # absolute deadline time (NaN if none)
#                 if np.isnan(feat["deadline_s"]):
#                     deadline_time = np.nan
#                 else:
#                     deadline_time = t_time + feat["deadline_s"]

#                 action_space_hint = "continuous" if feat["non_atomic"] == 1 else "discrete"

#                 rows_arrivals.append({
#                     "scenario": cfg.name,
#                     "episode_id": episode_id,
#                     "t_slot": t,
#                     "t_time": t_time,
#                     "agent_id": a.agent_id,
#                     "task_id": task_id_counter
#                 })

#                 rows_tasks.append({
#                     "scenario": cfg.name,
#                     "episode_id": episode_id,
#                     "task_id": task_id_counter,
#                     "agent_id": a.agent_id,
#                     "t_arrival_slot": t,
#                     "t_arrival_time": t_time,
#                     "b_mb": feat["b_mb"],
#                     "rho_cyc_per_mb": feat["rho"],
#                     "c_cycles": feat["c_cycles"],
#                     "mem_mb": feat["mem_mb"],
#                     "modality": feat["modality"],
#                     "has_deadline": feat["has_deadline"],
#                     "deadline_s": feat["deadline_s"],
#                     "deadline_time": deadline_time,
#                     "non_atomic": feat["non_atomic"],
#                     "split_ratio": feat["split_ratio"],
#                     "action_space_hint": action_space_hint
#                 })

#                 task_id_counter += 1

#     rows_episodes.append({
#         "scenario": cfg.name,
#         "episode_id": episode_id,
#         "Delta": Delta,
#         "T_slots": T_slots,
#         "hours": T_slots * Delta / 3600.0,
#         "N_agents": len(agents),
#         "seed": cfg.Episode.seed + episode_id
#     })

#     episodes_df = pd.DataFrame(rows_episodes)
#     agents_df   = pd.DataFrame(rows_agents)
#     arrivals_df = pd.DataFrame(rows_arrivals)
#     tasks_df    = pd.DataFrame(rows_tasks)

#     # Optimize dtypes (optional but useful)
#     if len(tasks_df):
#         tasks_df["modality"] = tasks_df["modality"].astype("category")
#         tasks_df["action_space_hint"] = tasks_df["action_space_hint"].astype("category")
#         # ints
#         for col in ["episode_id","task_id","agent_id","t_arrival_slot","has_deadline","non_atomic"]:
#             if col in tasks_df:
#                 tasks_df[col] = tasks_df[col].astype("int32")
#         # floats
#         for col in ["t_arrival_time","b_mb","rho_cyc_per_mb","c_cycles","mem_mb","deadline_s","deadline_time","split_ratio"]:
#             if col in tasks_df:
#                 tasks_df[col] = tasks_df[col].astype("float32")

#     if len(arrivals_df):
#         for col in ["episode_id","t_slot","agent_id","task_id"]:
#             if col in arrivals_df:
#                 arrivals_df[col] = arrivals_df[col].astype("int32")
#         for col in ["t_time"]:
#             if col in arrivals_df:
#                 arrivals_df[col] = arrivals_df[col].astype("float32")

#     if len(agents_df):
#         for col in ["agent_id"]:
#             agents_df[col] = agents_df[col].astype("int32")
#         for col in ["f_local","m_local","lam_sec"]:
#             agents_df[col] = agents_df[col].astype("float64")

#     return {
#         "episodes": episodes_df,
#         "agents":   agents_df,
#         "arrivals": arrivals_df,
#         "tasks":    tasks_df
#     }

# # -------------------------
# # save & plotting utilities
# # -------------------------
# def save_dataset(dfs: Dict[str, pd.DataFrame], out_dir: str = ".") -> Dict[str, str]:
#     os.makedirs(out_dir, exist_ok=True)
#     paths: Dict[str, str] = {}
#     for name, df in dfs.items():
#         csv_path = os.path.join(out_dir, f"{name}.csv")
#         df.to_csv(csv_path, index=False)
#         paths[name + "_csv"] = csv_path
#     return paths

# def _config_fingerprint(cfg: GlobalConfig) -> str:
#     s = json.dumps({
#         "scenario": cfg.name,
#         "Episode": asdict(cfg.Episode),
#         "AgentRanges": asdict(cfg.AgentRanges),
#         "TaskDist": asdict(cfg.TaskDist)
#     }, sort_keys=True).encode("utf-8")
#     return hashlib.sha256(s).hexdigest()[:16]

# def save_meta(cfg: GlobalConfig, out_dir: str = ".", qcap: float = 0.99) -> str:
#     meta = {
#         "schema_version": "1.0.0",
#         "scenario": cfg.name,
#         "seed": cfg.Episode.seed,
#         "fingerprint": _config_fingerprint(cfg),
#         "generated_at_utc": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
#         "env": {"python": platform.python_version(), "user": getpass.getuser()},
#         "Episode": asdict(cfg.Episode),
#         "N_agents": cfg.N_agents,
#         "AgentRanges": asdict(cfg.AgentRanges),
#         "TaskDist": asdict(cfg.TaskDist),
#         "units": {
#             "Delta": "seconds",
#             "lam_sec": "tasks per second (per agent)",
#             "per_slot_rate": "lam_sec * Delta",
#             "b_mb": "MB",
#             "rho_cyc_per_mb": "CPU cycles per MB",
#             "c_cycles": "CPU cycles",
#             "mem_mb": "MB",
#             "deadline_s": "seconds (relative); deadline_time = t_arrival_time + deadline_s",
#             "f_local": "Hz",
#             "m_local": "MB"
#         },
#         "notes": {
#             "policy_agnostic": True,
#             "queueing": "not simulated here",
#             "clipping": {
#                 "enabled": True,
#                 "method": "lognormal analytic quantile cap",
#                 "qcap": qcap,
#                 "z_table_keys": sorted(list(_Z_TABLE.keys()))
#             },
#             "action_space_hint": "derived from non_atomic; final decision belongs to environment"
#         }
#     }
#     path = os.path.join(out_dir, f"dataset_meta.json")
#     with open(path, "w", encoding="utf-8") as f:
#         json.dump(meta, f, ensure_ascii=False, indent=2)
#     return path

# def summarize_and_plot(dfs: Dict[str, pd.DataFrame], out_dir: str) -> None:
#     os.makedirs(out_dir, exist_ok=True)
#     tasks = dfs["tasks"].copy()
#     arrivals = dfs["arrivals"].copy()
#     episodes = dfs["episodes"].copy()

#     # ---- summary stats
#     def q(series: pd.Series, p: float):
#         return float(np.nanquantile(series.dropna(), p)) if len(series.dropna()) else float("nan")

#     tasks_per_hour = float(len(tasks)) / float(episodes.iloc[0]["hours"]) if len(episodes) and episodes.iloc[0]["hours"] > 0 else float("nan")

#     # modality distribution
#     mod_counts = tasks["modality"].value_counts(dropna=False) if len(tasks) else pd.Series(dtype=int)
#     mod_dist = {str(k): int(v) for k, v in mod_counts.to_dict().items()}

#     summary = {
#         "n_tasks": [len(tasks)],
#         "n_arrivals": [len(arrivals)],
#         "tasks_per_hour": [tasks_per_hour],
#         "b_mb_median": [q(tasks["b_mb"], 0.5)],
#         "b_mb_p90": [q(tasks["b_mb"], 0.9)],
#         "b_mb_p99": [q(tasks["b_mb"], 0.99)],
#         "rho_median": [q(tasks["rho_cyc_per_mb"], 0.5)],
#         "rho_p90": [q(tasks["rho_cyc_per_mb"], 0.9)],
#         "c_cycles_median": [q(tasks["c_cycles"], 0.5)],
#         "c_cycles_p90": [q(tasks["c_cycles"], 0.9)],
#         "c_cycles_p99": [q(tasks["c_cycles"], 0.99)],
#         "deadline_share": [float((tasks["has_deadline"]==1).mean()) if len(tasks) else float("nan")],
#         "non_atomic_share": [float((tasks["non_atomic"]==1).mean()) if len(tasks) else float("nan")],
#         "modality_counts_json": [json.dumps(mod_dist)]
#     }
#     pd.DataFrame(summary).to_csv(os.path.join(out_dir, f"summary_stats.csv"), index=False)

#     # ---- plots (each in its own figure)
#     def hist_plot(series: pd.Series, title: str, fname: str, logx: bool=False):
#         plt.figure()
#         s = series.dropna()
#         if len(s) == 0:
#             plt.title(title + " (no data)")
#         else:
#             plt.hist(s, bins=50)
#             if logx:
#                 plt.xscale('log')
#             plt.title(title)
#             plt.xlabel(title)
#             plt.ylabel("count")
#         plt.tight_layout()
#         plt.savefig(os.path.join(out_dir, fname), dpi=160)
#         plt.close()

#     hist_plot(tasks["b_mb"],            title="Task size (MB)",                fname=f"hist_b_mb.png", logx=True)
#     hist_plot(tasks["rho_cyc_per_mb"],  title="Compute density (cycles/MB)",   fname=f"hist_rho.png",  logx=True)
#     hist_plot(tasks["c_cycles"],        title="Total cycles",                  fname=f"hist_c_cycles.png", logx=True)
#     hist_plot(tasks["deadline_s"],      title="Deadline (s)",                  fname=f"hist_deadline_s.png", logx=False)
#     hist_plot(tasks.loc[tasks["non_atomic"]==1, "split_ratio"], title="Split ratio (only non-atomic)", fname=f"hist_split_ratio.png", logx=False)

#     # arrivals per agent
#     if len(arrivals):
#         per_agent = arrivals.groupby("agent_id").size()
#         plt.figure()
#         plt.bar(per_agent.index.astype(str), per_agent.values)
#         plt.title("Arrivals per agent")
#         plt.xlabel("agent_id")
#         plt.ylabel("count")
#         plt.tight_layout()
#         plt.savefig(os.path.join(out_dir, f"bar_arrivals_per_agent.png"), dpi=160)
#         plt.close()

# # -------------------------
# # scenario presets (light / moderate / heavy)
# # -------------------------
# HOURS = 1
# DEFAULT_DELTA = 1.0
# DEFAULT_T_SLOTS = int(HOURS * 3600 / DEFAULT_DELTA)

# BASE_EPISODE = EpisodeConf(Delta=DEFAULT_DELTA, T_slots=DEFAULT_T_SLOTS, seed=GLOBAL_SEED)
# BASE_AGENT_RANGES = AgentRanges(
#     lam_sec_min=0.02, lam_sec_max=0.80,
#     f_local_min=0.8e9, f_local_max=2.4e9,
#     m_local_min=3e3,  m_local_max=8e3 # MB
# )
# BASE_TASK_DIST = TaskFeatureDist()

# SCENARIOS: List[GlobalConfig] = [
#     GlobalConfig(
#         name="light",
#         N_agents=18,
#         Episode=replace(BASE_EPISODE, seed=GLOBAL_SEED + 101),
#         AgentRanges=replace(BASE_AGENT_RANGES, lam_sec_min=0.01, lam_sec_max=0.05),
#         TaskDist=replace(BASE_TASK_DIST,
#             b_median=2.0,  b_sigma_g=1.55,
#             rho_median=1.0e9, rho_sigma_g=1.45,
#             mem_median=64.0, mem_sigma_g=1.40,
#             p_deadline=0.15, deadline_min=0.8, deadline_max=2.0,
#             p_non_atomic=0.25, split_ratio_min=0.25, split_ratio_max=0.75)
#     ),
#     GlobalConfig(
#         name="moderate",
#         N_agents=18,
#         Episode=replace(BASE_EPISODE, seed=GLOBAL_SEED + 202),
#         AgentRanges=replace(BASE_AGENT_RANGES, lam_sec_min=0.05, lam_sec_max=0.20),
#         TaskDist=replace(BASE_TASK_DIST,
#             b_median=3.0,  b_sigma_g=1.60,
#             rho_median=1.2e9, rho_sigma_g=1.50,
#             mem_median=64.0, mem_sigma_g=1.45,
#             p_deadline=0.25, deadline_min=0.5, deadline_max=1.5,
#             p_non_atomic=0.35, split_ratio_min=0.30, split_ratio_max=0.80)
#     ),
#     GlobalConfig(
#         name="heavy",
#         N_agents=18,
#         Episode=replace(BASE_EPISODE, seed=GLOBAL_SEED + 303),
#         AgentRanges=replace(BASE_AGENT_RANGES, lam_sec_min=0.20, lam_sec_max=0.80),
#         TaskDist=replace(BASE_TASK_DIST,
#             b_median=5.0,  b_sigma_g=1.70,
#             rho_median=1.5e9, rho_sigma_g=1.55,
#             mem_median=64.0, mem_sigma_g=1.50,
#             p_deadline=0.35, deadline_min=0.3, deadline_max=1.0,
#             p_non_atomic=0.45, split_ratio_min=0.40, split_ratio_max=0.85)
#     )
# ]


# # -------------------------
# # main driver
# # -------------------------
# def main_generate(cfg: GlobalConfig, episodes: int = 1, out_root: str = "./datasets", qcap: float = 0.99) -> Dict[str, str]:
#     """Generate 'episodes' episodes for one scenario (fixed agent pool per scenario)."""
#     _validate_cfg(cfg)
#     out_dir = os.path.join(out_root, cfg.name)
#     os.makedirs(out_dir, exist_ok=True)

#     # build agents once per scenario to keep them consistent across its episodes
#     rng_agents = np.random.default_rng(cfg.Episode.seed + 10_000)
#     agents = build_agents(cfg, rng_agents)

#     all_paths: Dict[str, str] = {}
#     for ep in range(episodes):
#         ep_dir = os.path.join(out_dir, f"ep_{ep:03d}")
#         os.makedirs(ep_dir, exist_ok=True)

#         dfs = run_episode(cfg, agents, episode_id=ep, qcap=qcap)
#         paths = save_dataset(dfs, out_dir=ep_dir)
#         summarize_and_plot(dfs, out_dir=ep_dir)
#         all_paths.update({f"ep{ep}_{k}": v for k, v in paths.items()})

#     meta_path = save_meta(cfg, out_dir=out_dir, qcap=qcap)
#     all_paths["meta"] = meta_path
#     return all_paths

# def generate_all_scenarios(episodes_each: int = 1, out_root: str = "./datasets", qcap: float = 0.99) -> Dict[str, Dict[str, str]]:
#     results: Dict[str, Dict[str, str]] = {}
#     for cfg in SCENARIOS:
#         results[cfg.name] = main_generate(cfg, episodes=episodes_each, out_root=out_root, qcap=qcap)
#     return results

# if __name__ == "__main__":
#     out = generate_all_scenarios(episodes_each=1, out_root="./datasets")
#     print(json.dumps(out, indent=2))

{
  "light": {
    "ep0_episodes_csv": "./datasets\\light\\ep_000\\episodes.csv",
    "ep0_agents_csv": "./datasets\\light\\ep_000\\agents.csv",
    "ep0_arrivals_csv": "./datasets\\light\\ep_000\\arrivals.csv",
    "ep0_tasks_csv": "./datasets\\light\\ep_000\\tasks.csv",
    "meta": "./datasets\\light\\dataset_meta.json"
  },
  "moderate": {
    "ep0_episodes_csv": "./datasets\\moderate\\ep_000\\episodes.csv",
    "ep0_agents_csv": "./datasets\\moderate\\ep_000\\agents.csv",
    "ep0_arrivals_csv": "./datasets\\moderate\\ep_000\\arrivals.csv",
    "ep0_tasks_csv": "./datasets\\moderate\\ep_000\\tasks.csv",
    "meta": "./datasets\\moderate\\dataset_meta.json"
  },
  "heavy": {
    "ep0_episodes_csv": "./datasets\\heavy\\ep_000\\episodes.csv",
    "ep0_agents_csv": "./datasets\\heavy\\ep_000\\agents.csv",
    "ep0_arrivals_csv": "./datasets\\heavy\\ep_000\\arrivals.csv",
    "ep0_tasks_csv": "./datasets\\heavy\\ep_000\\tasks.csv",
    "meta": "./datasets\\heavy\\dataset_meta.json"
  }
